# Introduction to LoRA Tuning using PEFT from Hugging Face

<!-- ### Fine-tune a Foundational Model effortlessly -->


# LoRA Tuning

In this notebook you are being introduced to how to apply LoRA Tuning with the PEFT library to a pre-trained model.

For a complete list of Models compatible with PEFT refer to their [documentation](https://huggingface.co/docs/peft/main/en/index#supported-methods).

A short sample of models families available to be trained with PEFT are: Bloom, Llama, GPT-J, GPT-2, BERT... and more. Hugging Face is working hard to bring more Models to the Library.

## Brief introduction to LoRA Tuning.
LoRA is a re-parameterization technique. Its operation is simple, complex, and brilliant at the same time. It involves reducing the size of the matrices to be trained by dividing them in such a way that when multiplied, they yield the original matrix.

The weights that are modified are those of the reduced matrices, not the original matrix. It's better visualized in an image.

We have an original matrix of 50x50, which means we would have to modify about 2500 parameters. However, as we know, if we multiply two matrices of (2x50) and (50x2), we obtain a 50x50 matrix. Yet, these two matrices are formed by only 100 parameters each. In other words, for the reduced matrices, we need to modify a total of 200 parameters compared to the 2500 of the original matrix. This represents a 92% reduction, and the larger the original matrix, the greater the percentage of savings.

In Language Models like GPT-3 or any of the current ones with LoRA, it's possible that we only need to train about 0.02% of the original parameters. This varies for each model. The best part is that the obtained result is very similar to that of full fine-tuning, in some cases, it can even be better.

# Load the PEFT and Datasets Libraries.

The PEFT library contains the Hugging Face implementation of differente fine-tuning techniques, like LoRA Tuning.

Using the Datasets library we have acces to a huge amount of Datasets.

In [48]:
# !pip install -q peft==0.8.2
# !pip install -q datasets==2.16.1
#!pip install ipywidgets==7.7.5

From the transformers library we import the necesary classes to import the model and the tokenizer.

Then we can load the Tokenizer and the model.

Bloom is one of the smallest and smarter model available to be trained with PEFT Library using Prompt Tuning. You can use either of the models in the Bloom Family, I encorage you to use at least two of them and see the differences.

I'm using the smallest one just to spend less time trainig, and avoid memory problems in Colab.

In [49]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "bigscience/bloom-560m"
#model_name="bigscience/bloom-1b1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
foundation_model = AutoModelForCausalLM.from_pretrained(model_name)

## Inference with the pre-trained model.
I'm going to do a test with the pre-trained model without fine-tuning, to see if something changes after the fine-tuning.

In [50]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=100):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        repetition_penalty=1.5, #Avoid repetition.
        early_stopping=True, #The model can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id
    )
    return outputs

The dataset used for the fine-tuning contains prompts to be used with Large Language Models.

I'm going to request the pre-trained model that acts like a motivational coach.

In [51]:
#Inference original model
input_sentences = tokenizer("I want you to act as a motivational coach. ", return_tensors="pt")
foundational_outputs_sentence = get_outputs(foundation_model, input_sentences, max_new_tokens=50)

print(tokenizer.batch_decode(foundational_outputs_sentence, skip_special_tokens=False))

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


["I want you to act as a motivational coach.  You can do this by telling your students that they are not alone in their struggles, and then encouraging them.\nIf you're going through the same thing with someone else's kids or parents who have been struggling for years now (or even decades), it's time"]


Not sure if the answer is correct or not, but for sure is not a prompt. We need to train our model if we want that acts like a prompt engineer.

# Preparing the Dataset.
The Dataset used is:

https://huggingface.co/datasets/fka/awesome-chatgpt-prompts

In [52]:
import pandas as pd

df = pd.read_csv("hf://datasets/fka/awesome-chatgpt-prompts/prompts.csv")


df_tokenized = df.copy()
df_tokenized['input_ids'] = df['prompt'].apply(lambda x: tokenizer(x)['input_ids'])
df_tokenized['attention_mask'] = df['prompt'].apply(lambda x: tokenizer(x)['attention_mask'])

train_sample = df_tokenized.head(50)

train_sample = train_sample.drop('act', axis=1, errors='ignore')

display(train_sample)

,prompt,input_ids,attention_mask
0,Imagine you are an experienced Ethereum develo...,"[186402, 1152, 1306, 660, 72560, 28857, 167625...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"Using WebPilot, create an outline for an artic...","[39312, 15202, 51, 46712, 15, 7932, 660, 67606...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,I want you to act as a linux terminal. I will ...,"[44, 4026, 1152, 427, 1769, 661, 267, 104105, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"I want you to act as an English translator, sp...","[44, 4026, 1152, 427, 1769, 661, 660, 7165, 24...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,I want you to act as an interviewer. I will be...,"[44, 4026, 1152, 427, 1769, 661, 660, 33322, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,I want you to act as a javascript console. I w...,"[44, 4026, 1152, 427, 1769, 661, 267, 49760, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,I want you to act as a text based excel. you'l...,"[44, 4026, 1152, 427, 1769, 661, 267, 5484, 11...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,I want you to act as an English pronunciation ...,"[44, 4026, 1152, 427, 1769, 661, 660, 7165, 14...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,I want you to act as a spoken English teacher ...,"[44, 4026, 1152, 427, 1769, 661, 267, 93949, 7...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9,I want you to act as a travel guide. I will wr...,"[44, 4026, 1152, 427, 1769, 661, 267, 25008, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [53]:
eval_prompts = [
    "Your job is to act as a motivational coach.",
    "Explain photosynthesis to a child.",
    "Write a short poem about the ocean.",
    "Translate this to Spanish: How are you today?",
]


In [54]:
print(train_sample[:1])

                                              prompt  \
0  Imagine you are an experienced Ethereum develo...   

                                           input_ids  \
0  [186402, 1152, 1306, 660, 72560, 28857, 167625...   

                                      attention_mask  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  


# Fine-Tuning.
First is necesary create a LoRA config.


In [55]:
# TARGET_MODULES
# https://github.com/huggingface/peft/blob/39ef2546d5d9b8f5f8a7016ec10657887a867041/src/peft/utils/other.py#L220

import peft
from peft import LoraConfig, get_peft_model, PeftModel

lora_config = LoraConfig(
    r=4, #As bigger the R bigger the parameters to train.
    lora_alpha=1, # a scaling factor that adjusts the magnitude of the weight matrix. Usually set to 1
    target_modules=["query_key_value"], #You can obtain a list of target modules in the URL above.
    lora_dropout=0.05, #Helps to avoid Overfitting.
    bias="lora_only", # this specifies if the bias parameter should be trained.
    task_type="CAUSAL_LM"
)

The most important parameter is **r**, it defines how many parameters will be trained. As bigger the valuer more parameters are trained, but it means that the model will be able to learn more complicated relations between input and output.

Yo can find a list of the **target_modules** available on the [Hugging Face Documentation]( https://github.com/huggingface/peft/blob/39ef2546d5d9b8f5f8a7016ec10657887a867041/src/peft/utils/other.py#L220)

**lora_dropout** is like the commom dropout is used to avoid overfitting.

**bias** I was hesitating if use *none* or *lora_only*. For text classification the most common value is none, and for chat or question answering, *all* or *lora_only*.

**task_type**. Indicates the task the model is beign trained for. In this case, text generation.

### Create the PEFT model.



In [56]:
peft_model = get_peft_model(foundation_model, lora_config)
print(peft_model.print_trainable_parameters())

trainable params: 466,944 || all params: 559,607,808 || trainable%: 0.0834
None


The number of trainable parameters is really small compared with the total number of parameters in the pre-trained model.

In [57]:
#Create a directory to contain the Model
import os
working_dir = './'

output_directory = os.path.join(working_dir, "peft_lab_outputs")

In [58]:

from datasets import Dataset
train_dataset = Dataset.from_pandas(train_sample)


In the TrainingArgs we inform the number of epochs we want to train, the output directory and the learning_rate.

In [59]:
#Creating the TrainingArgs
import transformers
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir=output_directory,
    auto_find_batch_size=True, # Find a correct bvatch size that fits the size of Data.
    learning_rate= 3e-2, # it was 3e-2
    num_train_epochs=2, # it was 2
    report_to='none',
    use_cpu=True,
    label_names=['labels']
)

Now we can train the model.
To train the model we need:


*   The PEFT Model.
*   The training_args
* The Dataset
* The result of DataCollator, the Dataset ready to be procesed in blocks.





In [60]:
#This cell may take up to 15 minutes to execute.
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

Step,Training Loss


TrainOutput(global_step=14, training_loss=2.6231792994907925, metrics={'train_runtime': 102.2571, 'train_samples_per_second': 0.978, 'train_steps_per_second': 0.137, 'total_flos': 21871163621376.0, 'train_loss': 2.6231792994907925, 'epoch': 2.0})

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
#Save the model.
peft_model_path = os.path.join(output_directory, f"lora_model")

trainer.model.save_pretrained(peft_model_path)

In [63]:
#Load the Model.
loaded_model = PeftModel.from_pretrained(foundation_model,
                                        peft_model_path,
                                        is_trainable=False)

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


## Inference the fine-tuned model.

In [64]:
input_sentences = tokenizer("I want you to act as a motivational coach. ", return_tensors="pt")
foundational_outputs_sentence = get_outputs(loaded_model, input_sentences, max_new_tokens=50)

print(tokenizer.batch_decode(foundational_outputs_sentence, skip_special_tokens=True))

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


['I want you to act as a motivational coach.  You will provide me with information about someone who is looking for help in finding their motivation and goals. I can also suggest that they should use some of the same techniques used by other people, such as: talking them through strategies or giving advice on how best']


The result is amazing. Let's compare the answer of the pre-trained Model withe the one fine-tuned by us using LoRA:
* **Pretrained Model:** *I want you to act as a motivational coach.*  Don't be afraid of being challenged.
* **Fine-Tuned Model:** I want you to act as a motivational coach.  I will provide some information about someone\'s motivation and goals, but it should be your job  in order my first request – "I need someone who can help me find the best way for myself stay motivated when competing against others." My suggestion is “I have

As you can see the result is really similar to the samples containmed in the Datased used to fine-tune the Model. And we only trained the Model for 10 epochs and with a really small number of rows.

# Continue Learning
Please, play with all the variables in the notebook and drive your own experiments and get your conclusions.

Try to change the **lora_config** values, maybe you can achieve a better result in less epochs, saving time and money for your company. :-)


In [65]:
def evaluate_model_output(model, tokenizer, prompts):
    for prompt in prompts:
        inputs = tokenizer(prompt, return_tensors="pt")
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=50,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7
        )
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"\n🧪 Prompt: {prompt}\n💬 Output: {decoded}")


In [66]:
print("🔹 Before LoRA Fine-Tuning:")
evaluate_model_output(foundation_model, tokenizer, eval_prompts)

print("\n🔸 After LoRA Fine-Tuning:")
evaluate_model_output(loaded_model, tokenizer, eval_prompts)


🔹 Before LoRA Fine-Tuning:

🧪 Prompt: Your job is to act as a motivational coach.
💬 Output: Your job is to act as a motivational coach. I will provide you with information on someone who wants to improve their own life style and lifestyle habits. I will also provide you with information on their family members and your role is to help them improve their lifestyle habits. You will also provide you with my

🧪 Prompt: Explain photosynthesis to a child.
💬 Output: Explain photosynthesis to a child. Ask them what the photosynthesis is. They can use photosynthesis as a metaphor for living organisms. I will provide a series of photosynthesis experiments that demonstrate the ability of plants to capture energy from sunlight. You can also use photosynthesis as a metaphor

🧪 Prompt: Write a short poem about the ocean.
💬 Output: Write a short poem about the ocean. The theme should be your own experience. The poem should also contain words related to the theme. It should also be a poem that capture